# 01 — Data Exploration 
- Tải `AB_NYC_2019.csv` (nếu chưa có) bằng Kaggle CLI
- Đọc CSV *không dùng pandas* (chỉ csv + numpy) qua `src.data_processing`
- In report: shape, cột quan trọng, unique của nhóm/kind, NA, phạm vi availability_365

In [ ]:
# thêm project root (thư mục chứa 'src/') vào sys.path
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").is_dir() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

# import hàm để load dữ liệu
from src import load_and_check 

## 1. Tải & kiểm tra dữ liệu

In [ ]:
report = load_and_check(root=str(ROOT / "data" / "raw"))